In [3]:
pip install transformers torch


In [1]:
from glob import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline
from nltk import sent_tokenize
from tqdm import tqdm



## Loaded the subtitles data

In [6]:
file_paths = sorted(glob("C:/Users/Wen/OneDrive/Documents/GitHub/Portfolio_Project/Deep Learning/Shinobi Naruto Insights -NLP/data/Naruto Complete Subtitles Series/*.ass"))

scripts = []
number_episodes = []

for path in file_paths:
    try:
        with open(path, "r") as file:
            lines = file.readlines()
            lines = lines[27:]
            rows = [",".join(line.split(",")[9:]) for line in lines if line.startswith('Dialogue:')]
            rows = [line.replace("\\N", " ") for line in rows]
            script = " ".join(rows)
            episode = int(os.path.basename(path).split("-")[1].split(".")[0].strip())
            scripts.append(script)
            number_episodes.append(episode)
    except Exception as e:
        print(f"Failed to process {path}: {str(e)}")

In [7]:
df = pd.DataFrame.from_dict({'Episodes': number_episodes, "script":scripts})
df.head()

,Episodes,script
0,1,"A long time ago, a powerful demon fox appeared..."
1,2,"C'mon!\n Running like a fugitive,\n Being chas..."
2,3,"C'mon!\n Running like a fugitive,\n Being chas..."
3,4,"C'mon!\n Running like a fugitive,\n Being chas..."
4,5,"C'mon!\n Running like a fugitive,\n Being chas..."


## Using Zero-Shot Classification from HuggingFace

Zero-shot text classification is a task in natural language processing where a model is trained on a set of labeled examples but is then able to classify new examples from previously unseen classes.


In [8]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

themes = ["friendship", "rivalry", "adventure", "redemption", "training", "conflict", "love", "betrayal", "growth"]

text = "Naruto and Sasuke once again face each other on the battlefield, their rivalry reaching a peak."

results = classifier(text, candidate_labels=themes, multi_label=True)

print("Labels:", results['labels'])
print("Scores:", results['scores'])


Labels: ['conflict', 'rivalry', 'growth', 'adventure', 'betrayal', 'training', 'redemption', 'friendship', 'love']
Scores: [0.9989086389541626, 0.998081624507904, 0.23836888372898102, 0.08800016343593597, 0.006903242785483599, 0.000422329583670944, 0.00011362966324668378, 7.620819815201685e-05, 4.696832911577076e-05]


In [14]:
def get_themes(script):
    script_sentences = sent_tokenize(script)

    script_batches = []
    sentence_batch_size=1000
    for index in range(0,len(script_sentences),sentence_batch_size):
        sent = " ".join(script_sentences[index:index+sentence_batch_size])
        script_batches.append(sent)
    
    candidate_labels = ["friendship", "rivalry", "adventure", "redemption", "training", "conflict", "love", "betrayal", "growth"]
    emotion_output = classifier(script_batches, 
                       candidate_labels,multi_label=True)

    emotions={}
    for output in  emotion_output:
        for label, score in zip(output['labels'],output['scores']):
            if label not in emotions:
                emotions[label]=[]
            emotions[label].append(score)
    
    emotions =  {key:np.mean(np.array(value)) for key,value in emotions.items()}
   
    return emotions


In [15]:
tqdm.pandas()
output_emotions = df['script'].progress_apply(get_themes)


  0%|          | 0/218 [00:00<?, ?it/s]

  1%|          | 2/218 [02:18<4:08:48, 69.11s/it]


KeyboardInterrupt: 

In [ ]:
emotion_df = pd.DataFrame(output_emotions.tolist())


NameError: name 'output_emotions' is not defined